In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotion-dataset/validation.csv
/kaggle/input/emotion-dataset/training.csv
/kaggle/input/emotion-dataset/test.csv


In [3]:
import torch
from transformers import BertTokenizer,BertModel

import logging

import matplotlib.pyplot as plt
%matplotlib inline

tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
import pandas as pd 
import numpy as np

dataframe=pd.read_csv('/kaggle/input/emotion-dataset/training.csv')
test_data=pd.read_csv('/kaggle/input/emotion-dataset/test.csv')
val_data=pd.read_csv('/kaggle/input/emotion-dataset/validation.csv')

In [5]:
df=dataframe['text']

In [10]:
def input_add(x):
    return "[CLS]"+" "+x+" "+"[SEP]"

df=df.apply(input_add)

In [27]:
text=df.tolist()
print(text[1])
i=0
token2=[]
for x in text:
        token2.append(tokenizer.tokenize(text[i]))
        i+=1

        
print(token2)

[CLS] [CLS] i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake [SEP] [SEP]
[['[CLS]', '[CLS]', 'i', 'didn', '##t', 'feel', 'humiliated', '[SEP]', '[SEP]'], ['[CLS]', '[CLS]', 'i', 'can', 'go', 'from', 'feeling', 'so', 'hopeless', 'to', 'so', 'damned', 'hopeful', 'just', 'from', 'being', 'around', 'someone', 'who', 'cares', 'and', 'is', 'awake', '[SEP]', '[SEP]'], ['[CLS]', '[CLS]', 'im', 'grabbing', 'a', 'minute', 'to', 'post', 'i', 'feel', 'greedy', 'wrong', '[SEP]', '[SEP]'], ['[CLS]', '[CLS]', 'i', 'am', 'ever', 'feeling', 'nos', '##tal', '##gic', 'about', 'the', 'fireplace', 'i', 'will', 'know', 'that', 'it', 'is', 'still', 'on', 'the', 'property', '[SEP]', '[SEP]'], ['[CLS]', '[CLS]', 'i', 'am', 'feeling', 'gr', '##ou', '##chy', '[SEP]', '[SEP]'], ['[CLS]', '[CLS]', 'iv', '##e', 'been', 'feeling', 'a', 'little', 'burden', '##ed', 'lately', 'wasn', '##t', 'sure', 'why', 'that', 'was', '[SEP]', '[SEP]'], ['[CLS]', '[CLS]', 'i

In [ ]:
df.head()

In [ ]:
#indexed_token=tokenizer.convert_tokens_to_ids(token)

In [ ]:

from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

#Function to generate BERT embeddings for a given text
def get_bert_embeddings(sentences):
    tokenized_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = bert_model(**tokenized_input)
        embeddings = model_output.last_hidden_state.mean(dim=1).numpy() 
    return embeddings

In [ ]:
def embeddings(text):
    tokenized_text=tokenizer.tokenize(text)
    indexed_tokens=tokenizer.convert_tokens_to_ids(tokenized_text)
   
    segmentids=[1]*len(tokenized_text)
    
    
    #convert inputs to pytorch tensors
    tokens_tensor=torch.tensor([indexed_tokens])
    segments_tensors=torch.tensor([segmentids])
    
    model=BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)
    model.eval()
    with torch.no_grad():
        outputs=model(tokens_tensor,segments_tensors)
        hidden_states=outputs[2]
    
    token_embeddings=torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    
    # Stores the token vectors, with shape [22 x 3,072]
    token_vecs_cat = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Concatenate the vectors (that is, append them together) from the last 
        # four layers.
        # Each layer vector is 768 values, so `cat_vec` is length 3,072.
        cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

        # Use `cat_vec` to represent `token`.
        token_vecs_cat.append(cat_vec)
    
    
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)

        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)
    
    # `token_vecs` is a tensor with shape [22 x 768]
    token_vecs = hidden_states[-2][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return sentence_embedding

In [ ]:
embeddings=df.apply(get_bert_embeddings)

In [ ]:
print(len(embeddings[1][0]))

In [ ]:
df['embeddings']=df.apply(embeddings)

In [ ]:
print(df['embeddings'])


In [ ]:
from sklearn.model_selection import train_test_split
label=dataframe['label']
 


In [ ]:
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV 

param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': ['scale',
                       'auto'], 
              'kernel': ['linear','rbf']}  
model=SVC()
grid = GridSearchCV(model, param_grid, cv = 3,scoring='accuracy') 
grid.fit(embeddings, label)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_imdb["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_imdb["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_imdb["test"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="model_",
    tokenizer=tokenizer,
)

In [ ]:
callbacks = [metric_callback, push_to_hub_callback]

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="stevhliu/my_awesome_model")
classifier(text)

# roBERTa

In [24]:
from transformers import RobertaTokenizer, RobertaModel
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import torch
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score

In [29]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

train_data=dataframe
data = {'Train Data': train_data, 'Validation Data': val_data, 'Test Data': test_data}

In [28]:
pip install imblearn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 4.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
train_x, train_y = ros.fit_resample(np.array(train_data['text']).reshape(-1, 1), np.array(train_data['label']).reshape(-1, 1))
train = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['text', 'label'])

In [31]:
from sklearn import preprocessing
le = preprocessing.OneHotEncoder()
y_train= le.fit_transform(np.array(train['label']).reshape(-1, 1)).toarray()
y_test= le.fit_transform(np.array(test_data['label']).reshape(-1, 1)).toarray()
y_val= le.fit_transform(np.array(val_data['label']).reshape(-1, 1)).toarray()

In [33]:
def roberta_encode(data,maximum_length) :
  input_ids = []
  attention_masks = []
  

  for i in range(len(data.text)):
      encoded = tokenizer.encode_plus(
        
        data.text[i],
        add_special_tokens=True,
        max_length=maximum_length,
        pad_to_max_length=True,
        truncation=True,
        return_attention_mask=True,
        
      )
      
      input_ids.append(encoded['input_ids'])
      attention_masks.append(encoded['attention_mask'])
  return np.array(input_ids),np.array(attention_masks)

In [34]:
max_len = max([len(x.split()) for x in train_data['text']])
train_input_ids,train_attention_masks = roberta_encode(train, max_len)
test_input_ids,test_attention_masks = roberta_encode(test_data, max_len)
val_input_ids,val_attention_masks = roberta_encode(val_data, max_len)

/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [40]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

def create_model(bert_model, max_len):
    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')

    output = bert_model([input_ids,attention_masks])
    output = output[1]

    output = tf.keras.layers.Dense(6, activation='softmax')(output)
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks],outputs = output)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [41]:
from transformers import TFRobertaModel
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'roberta.embeddings.position_ids', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [42]:
model = create_model(roberta_model, max_len)
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 66)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 66)]                 0         []                            
                                                                                                  
 tf_roberta_model_2 (TFRobe  TFBaseModelOutputWithPooli   1246456   ['input_5[0][0]',             
 rtaModel)                   ngAndCrossAttentions(last_   32         'input_6[0][0]']             
                             hidden_state=(None, 66, 76                                           
                             8),                                                            

In [ ]:
history = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_val), epochs=3,batch_size=100)

Epoch 1/3
322/322 [==============================] - 1662s 5s/step - loss: 0.5942 - accuracy: 0.7742 - val_loss: 0.2209 - val_accuracy: 0.9230
Epoch 2/3
261/322 [=======================>......] - ETA: 5:03 - loss: 0.1637 - accuracy: 0.9389

# bert

In [6]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [7]:
df=dataframe

In [8]:

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby([ 'label', 'data_type']).count()

text
label data_type      
0     train      3966
      val         700
1     train      4558
      val         804
2     train      1108
      val         196
3     train      1835
      val         324
4     train      1647
      val         290
5     train       486
      val          86

In [10]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    truncation=True,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)



/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2618: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [11]:

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=6,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [13]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),no_deprecation_warning=True,
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 3

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [14]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    #label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
       # print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


In [ ]:

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    
    tqdm.write(f'\nEpoch {epoch}')
    
    apc=accuracy_per_class(predictions, true_vals)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'accuracy: {apc}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/4534 [00:00<?, ?it/s]

# text classsification using bert model from huggingface

In [65]:
from transformers import TFAutoModel,AutoTokenizer
from datasets import load_dataset

In [2]:
model=TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [3]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [4]:
input=tokenizer(["hello world","hi i am starting bert"],padding=True,truncation=True,return_tensors='tf')
input

{'input_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[  101,  7592,  2088,   102,     0,     0,     0],
       [  101,  7632,  1045,  2572,  3225, 14324,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 7), dtype=int32, numpy=
array([[1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [5]:
output=model(input)
output


TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(2, 7, 768), dtype=float32, numpy=
array([[[-0.16888388,  0.1360631 , -0.13940057, ..., -0.6251126 ,
          0.05217275,  0.3671454 ],
        [-0.3632752 ,  0.14121905,  0.879988  , ...,  0.10433033,
          0.28875685,  0.37267917],
        [-0.69859445, -0.69879746,  0.0645021 , ..., -0.22103682,
          0.00986898, -0.5939794 ],
        ...,
        [-0.40266657, -0.01928248,  0.5732505 , ..., -0.20656855,
          0.0233856 ,  0.20126322],
        [-0.62284076, -0.27453515,  0.18117645, ..., -0.12944834,
         -0.03839098, -0.05733182],
        [-0.30099404, -0.1347453 ,  0.5684589 , ..., -0.20533222,
          0.00306507,  0.15375361]],

       [[ 0.14302444,  0.3074709 ,  0.18806851, ..., -0.13069361,
          0.13434792,  0.18480325],
        [ 0.38674432,  0.26724976,  1.1220928 , ...,  0.02218601,
          0.121971  , -0.37112597],
        [ 0.08931706, -0.6947453 ,  0.4987764 , ...

In [6]:

pip install -U datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [7]:
from datasets import load_dataset

emotions = load_dataset('SetFit/emotion')

/opt/conda/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
emotions

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 2000
    })
})

In [10]:
def tokenize(batch):
    return tokenizer(batch['text'],padding=True,truncation=True)

In [11]:
emotion_encoded=emotions.map(tokenize,batch_size=None,batched=True)
emotion_encoded

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [12]:
BATCH_SIZE=64
import tensorflow as tf
def order(inp):
    data=list(inp.values())
    return {
        'input_ids':data[3],
        'attention_mask':data[4],
        'token_type_ids':data[5]
    },data[1]

train_dataset=tf.data.Dataset.from_tensor_slices(emotion_encoded['train'][:])
train_dataset=train_dataset.batch(BATCH_SIZE).shuffle(1000)

train_dataset=train_dataset.map(order,num_parallel_calls=tf.data.AUTOTUNE)

test_dataset=tf.data.Dataset.from_tensor_slices(emotion_encoded['test'][:])
test_dataset=test_dataset.batch(BATCH_SIZE)
test_dataset=test_dataset.map(order,num_parallel_calls=tf.data.AUTOTUNE)

In [13]:
inp,out=next(iter(train_dataset))
print(inp,'\n\n',out)

{'input_ids': <tf.Tensor: shape=(64, 87), dtype=int32, numpy=
array([[ 101, 1045, 3651, ...,    0,    0,    0],
       [ 101, 1045, 2514, ...,    0,    0,    0],
       [ 101, 1045, 2514, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2572, ...,    0,    0,    0],
       [ 101, 1045, 2134, ...,    0,    0,    0],
       [ 101, 1045, 2514, ...,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(64, 87), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(64, 87), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>} 

 tf.Tensor(
[2 0 0 0 1 4 1 4 3 2 4 3

In [15]:
class BERTForClassification(tf.keras.Model):
    def __init__(self,bert_model,num_classes):
        super().__init__()
        self.bert=bert_model
        self.fc=tf.keras.layers.Dense(num_classes,activation='softmax')
    def call(self,inputs):
        x=self.bert(inputs)[1]
        return self.fc(x)


In [16]:
classifier=BERTForClassification(model,num_classes=6)
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'])

In [17]:
history=classifier.fit(train_dataset,
                      epochs=3
                     )

Epoch 1/3
250/250 [==============================] - 7759s 31s/step - loss: 1.1070 - accuracy: 0.5789
Epoch 2/3
250/250 [==============================] - 7711s 31s/step - loss: 0.3143 - accuracy: 0.8910
Epoch 3/3
250/250 [==============================] - 7718s 31s/step - loss: 0.1654 - accuracy: 0.9342


In [ ]:
classifier.evaluate(test_dataset)

# using distilbert from hugging face

In [11]:
import os
import ktrain
from ktrain import text


ModuleNotFoundError: No module named 'ktrain'

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [13]:
text=dataframe['text'].tolist()
label=dataframe['label'].tolist()

In [15]:
#model building
import torch
from transformers import DistilBertTokenizer,AutoTokenizer
from transformers import DistilBertForSequenceClassification,AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [16]:
train_text,val_texts,train_labels,val_labels=train_test_split(text,label,test_size=.2,random_state=42)

In [17]:
checkpoint='distilbert-base-uncased'
tokenizer=DistilBertTokenizer.from_pretrained(checkpoint)
model=DistilBertForSequenceClassification.from_pretrained(checkpoint,num_labels=6,problem_type='multi_class_classification')

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
#custom dataset
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __inti__(self,text,label,tokenizer,max_len=128):
    
        self.texts=text
        self.labels=label
        self.tokenizer=tokenizer
        self.max_len=max_len
        
        
    def __len__(self):
        return len(self.texts)
    
    
    def __getitem__(self,idx):
        text=str(self.texts[idx])
        label=torch.tensor(self.label[idx])
        encoding=self.tokenizer(text,truncation=True,padding='max_lenght',max_length=self.max_len,return_tensors='pt')
        return {
            'inputs_ids':encoding['input_ids'].flatten(),
            'attention_mask':encoding['attention_mask'].flatten(),
            'labels':label
            }

In [20]:
train_dataset= CustomDataset(val_texts,val_labels,tokenizer)
val_dataset= CustomDataset(val_texts,val_labels,tokenizer)

TypeError: CustomDataset() takes no arguments

In [ ]:
val_dataset[0]